# McStasScript
McStasScript allows writing and running McStas simulations directly from python.

Written by Mads Bertelsen while working at ESS DMSC.

Funded by PaNOSC and a part of PaNOSC work package 5.

Documentation online: https://mads-bertelsen.github.io

### Installation
McStasScript is open source and available here: https://github.com/PaNOSC-ViNYL/McStasScript

Easy to install, install McStas as normal and then McStasScript can be installed with:
```
pip install McStasScript --upgrade
```

![normal](figures/Slide1.png)

![McStasScript](figures/Slide2.png)

## Why move to Python?
- McStas can still be used as usual, McStasScript is just an alternative
- New generation of scientists are more likely to know Python
- One language for writing the simulation and analysis of data

## Why have a software layer that manages McStas things?
- Provides powerful levels of abstraction (functions, for loops, ...)
- Can in many cases find problems earlier than when running the instrument
- Can host relevant McStas tools

# Live demo of McStasScript
Here we will set up an instrument simulation together and execute it from the slides!
First we import the package.

In [ ]:
import mcstasscript as ms

## Configuration
Before using McStasScript for the first time, it needs to be configured to find your McStas and/or McXtrace installation.

In [ ]:
configurator = ms.Configurator()
configurator.set_mcrun_path("/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/")
configurator.set_mcstas_path("/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1")
print(configurator)

## Create instrument objects
The main class is the McStas_instr, used for creating instrument objects.

In [ ]:
instrument = ms.McStas_instr("demo", author="Mads Bertelsen")

## Help from the instrument object
The instrument object knows about your available McStas components, and has help methods available.

In [ ]:
instrument.available_components()

In [ ]:
instrument.available_components("sources")

In [ ]:
instrument.component_help("Moderator")

## Adding components
McStas simulations rely on a sequence of components, these can be added to the instrument object with the *add_component* method. These return an instrument object that can be used to modify the component further.

In [ ]:
src = instrument.add_component("source", "Source_div")
instrument.show_components()

In [ ]:
print(src)

The component parameters become python attributes on the component object.

In [ ]:
src.xwidth = 0.1
src.yheight = 0.05
src.focus_aw = 1.2
src.focus_ah = 2.3
src.set_parameters(xwidth=0.09, yheight=0.04)
print(src)

Notice the attributes on component objects auto complete and wrong attributes are not allowed.

In [ ]:
src.focus_ah = 2.2

### All information about a component
The *show_parameters* method is good for getting an overview of the current state of the component parameters, as it will also show the default values.

In [ ]:
src.show_parameters()

## Add instrument parameters
Its also possible to add instrument parameters, and these can be used directly when setting component parameters.

In [ ]:
instrument.add_parameter("int", "order", value=1, comment="Monochromator order, integer")
instrument.show_parameters()

In [ ]:
src.lambda0 = instrument.add_parameter("wavelength", value=2.0, comment="Wavelength in [Ang]")
src.dlambda = "0.01*wavelength"
print(src)

### Setting component positions
When creating a new component, one can specify its position and rotation. 

In [ ]:
guide = instrument.add_component("guide", "Guide_gravity", AT=[0,0,1], RELATIVE=src)

In [ ]:
guide.set_parameters(w1=0.05, h1=0.05, l=8.0, m=3.5, G=-9.82)
print(guide)

One can always set the positon and rotation of component objects with the appropriate methods at a later point.

In [ ]:
guide.set_AT([0, 0, 2], RELATIVE="source")

### Adding initialize code
Its possible to add lines of C code to the initialize section of the written McStas file, but its equally possible to just perform these calculations directly in python. Here I show how calculate rotation of a monochromator before we add such a component.

In [ ]:
instrument.add_declare_var("double", "mono_Q", value=2*3.14159/1.714) # Q for Ge 311
instrument.add_declare_var("double", "wavevector")
instrument.append_initialize("wavevector = 2.0*PI/wavelength;")

instrument.add_declare_var("double", "mono_rotation")
instrument.append_initialize("mono_rotation = asin(mono_Q/(2.0*wavevector))*RAD2DEG;")

In [ ]:
instrument.show_variables()

### Adding a monochromator
We can now add a monochromator that use the calculated rotation.

In [ ]:
mono = instrument.add_component("mono", "Monochromator_flat")

In [ ]:
mono.set_parameters(zwidth=0.05, yheight=0.08, Q="mono_Q")
mono.set_AT([0, 0, guide.l + 0.5], RELATIVE=guide)
mono.set_ROTATED([0, "mono_rotation", 0], RELATIVE="guide")

print(mono)

Need to set the beam direction to the direction of the scattered beam

In [ ]:
beam_direction = instrument.add_component("beam_dir", "Arm", AT_RELATIVE=mono)
beam_direction.set_ROTATED([0, "mono_rotation", 0], RELATIVE=mono)

### Adding a sample
Now we can set up a simple powder sample to get some mildly interesting data.

In [ ]:
sample = instrument.add_component("sample", "PowderN", AT=[0,0,1.1], RELATIVE="beam_dir")

In [ ]:
sample.radius = 0.015
sample.yheight = 0.05
sample.reflections = '"Na2Ca3Al2F14.laz"'
print(sample)

### Adding a few monitors
At the end we add a few monitors, a two theta banana detector and a transmission PSD.

In [ ]:
banana = instrument.add_component("banana", "Monitor_nD", RELATIVE=sample)
banana.xwidth = 2.0
banana.yheight = 0.3
banana.restore_neutron = 1
banana.filename = '"banana.dat"'
banana.options = '"theta limits=[5 175] bins=150, banana"'

mon = instrument.add_component("monitor", "PSD_monitor")
mon.nx = 100
mon.ny = 100
mon.filename = '"psd.dat"'
mon.xwidth = 0.05
mon.yheight = 0.08
mon.restore_neutron = 1
mon.set_AT([0,0,0.1], RELATIVE=sample)

### Checking our instrument object
Before running our simulation we can check the contents of the instrument object and ensure its reasonable.

In [ ]:
instrument.show_components()

In [ ]:
instrument.show_parameters()

### Instrument diagram
Instrument diagram can be generated to provide an overview of the instrument.

In [ ]:
%matplotlib widget
instrument.show_diagram()

## Visualizing the instrument
Can visualize the instrument directly from the notebook

In [ ]:
instrument.show_instrument(width=1000,height=800)

### Runnig our simulation
Now we can run the simulation directly from python.

In [ ]:
instrument.settings(ncount=5E6, output_path="data_folder/mcstas_basics")
instrument.set_parameters(wavelength=2.8)
data = instrument.backengine()

### Plotting the data
The *backengine* method returns a data object that we can plot.

In [ ]:
print(data)

In [ ]:
ms.make_sub_plot(data, fontsize=16, figsize=(15, 6))

### Plotting interface
A widget interface is also available for plotting.

In [ ]:
%matplotlib widget
import mcstasscript.jb_interface as ms_widget
ms_widget.show(data)

### Simulation interface

In [ ]:
ms_widget.show(instrument)

### Setting plotting options
Its also possible to adjust the way data is plotted, and these preferences are stored in the data object.

In [ ]:
ms.name_plot_options("banana", data, left_lim=90, right_lim=150)
ms.name_plot_options("monitor", data, log=True, orders_of_mag=3)
ms.make_sub_plot(data, fontsize=12, figsize=(9, 4))

### Access to the underlying data
The underlying data is read from the McStas data files and imported as numpy arrays.

In [ ]:
import numpy as np
banana = ms.name_search("banana", data)
print(banana)

In [ ]:
print(banana.Intensity[1:10])

### Plot with matplotlib

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7,4))
plt.plot(banana.xaxis, banana.Intensity)
plt.xlabel("Two theta [deg]")
plt.ylabel("Intensity [n/s/bin]")
plt.show()

## Changing the instrument
The most common way to create the instrument is to add components in a sequence, but it is possible to insert a component in another position in the instrument. Lets add a slit and monitor between the monochromator and sample.

In [ ]:
instrument.show_diagram()

### Inserting a component before or after another
When adding a component one can specify the before or after keyword to select the location.

In [ ]:
instrument.get_component("sample")

In [ ]:
slit = instrument.add_component("sample_slit", "Slit", before="sample")
slit.radius = 0.01
slit.set_AT(1.0, RELATIVE="beam_dir")

mon = instrument.add_component("sample_monitor", "PSD_monitor", after=slit)
mon.set_parameters(xwidth=0.021, yheight=0.021,
                   restore_neutron=1, filename='"sample_psd"')
mon.set_AT([0,0,0.01], RELATIVE=slit)

In [ ]:
instrument.show_components()

## Run with the slit

In [ ]:
ms_widget.show(instrument)

## Moving a component
Components can be moved within the component sequence, for example we could switch the position of the monitor and slit.

In [ ]:
instrument.show_components()

In [ ]:
instrument.move_component("sample_slit", after="sample_monitor")

### Checking the instrument for errors
Some operations checks for errors, for example moving a component, one can also do it manually and get the error message.

In [ ]:
try:
    instrument.check_for_errors()
except Exception as e:
    print(e)

### Fixing the instrument

In [ ]:
instrument.show_components()

In [ ]:
mon.set_AT(1.0, RELATIVE="beam_dir")
slit.set_AT(0.01, RELATIVE=mon)

In [ ]:
try:
    instrument.check_for_errors()
except Exception as e:
    print(e)

### Running the instrument to check
The sample monitor is now in front of the slit and everything should be working again.

In [ ]:
ms_widget.show(instrument)

## Removing a component
Use the *remove_component* to remove a component from the instrument.

In [ ]:
instrument.remove_component(slit)
instrument.remove_component("sample_monitor")
ms_widget.show(instrument)

## Final remarks
McStasScript provides a new way to use McStas from python!

A full tutorial exist as Jupyter Notebooks, this presentation was a condensed version of the first part called McStas_basics.
The tutorial covers both how to use McStasScript and the Union components.

Documentaton and tutorials can be found here [https://mads-bertelsen.github.io](https://mads-bertelsen.github.io)

## Exercise

On the e-learning platform go to the jupyter lab and navigate to the McStas Advanved School day 1 directory, here you will find this talk as a jupyter notebook and todays exercises along with solutions. After this talk you are ready for exercise 1!